<a href="https://colab.research.google.com/github/dannezri/script_planning_check/blob/main/planning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import requests
import os
from datetime import datetime, timedelta


# Mapping des centres
centres_mapping = {
    0: "PSS",
    1: "ELB",
    2: "ASS",
    3: "GON",
    4: "PON"
}

# Mapping des niveaux
niveau_mapping = {
    "CP": 0, "CE1": 1, "CE2": 2, "CM1": 3, "CM2": 4,
    "6ème": 5, "5ème": 6, "4ème": 7, "3ème": 8,
    "2nd": 9, "1ère": 10, "Term": 11
}


# Calcul des dates dynamiques
date_debut = datetime.today().strftime('%Y-%m-%d')
date_fin = (datetime.today() + timedelta(days=7)).strftime('%Y-%m-%d')

# Inversion du mapping pour retrouver la classe depuis son niveau
inverse_niveau_mapping = {v: k for k, v in niveau_mapping.items()}

# URL pour déclencher l'extraction du planning avec les dates dynamiques
extraction_url = f"https://planning.geniusclass.fr/request/extraction_planning.php?date_debut={date_debut}&date_fin={date_fin}"
json_url = "https://planning.geniusclass.fr/request/templates/combined.json"

# Déclencher la génération du JSON
print("🔄 Extraction du planning en cours...")
requests.get(extraction_url)

# Télécharger le fichier JSON
print("🔽 Téléchargement du fichier JSON...")
response = requests.get(json_url)
if response.status_code == 200:
    file_path = "combined.json"
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(response.text)
    print("✅ Fichier JSON téléchargé avec succès.")
else:
    print("❌ Erreur lors du téléchargement du fichier JSON.")
    exit()

# Charger le JSON
def load_json(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        return json.load(file)

data = load_json(file_path)

# Fonction de validation du planning
def validate_schedule(data):
    errors = []

    for centre in data["centres"]:
        centre_id = centre["idCentre"]
        centre_nom = centres_mapping.get(centre_id, f"Inconnu ({centre_id})")

        for date, schedule in centre["dates"].items():
            for heure, session in schedule["heures"].items():
                tuteurs = session.get("tuteurs", [])

                if len(tuteurs) > 3:
                    errors.append({
                        "centre": centre_nom,
                        "date": date,
                        "heure": heure,
                        "erreur": "Plus de 3 groupes simultanés",
                        "type_erreur": "Organisation"
                    })

                for tuteur in tuteurs:
                    tuteur_prenom = tuteur["prenom"]
                    tuteur_nom = tuteur["nom"]

                    if "matieres_dispensees" not in tuteur or not tuteur["matieres_dispensees"]:
                        errors.append({
                            "centre": centre_nom,
                            "date": date,
                            "heure": heure,
                            "tuteur": f"{tuteur_prenom} {tuteur_nom}",
                            "erreur": f"{tuteur_prenom} {tuteur_nom} n'a pas de matières définies.",
                            "type_erreur": "Configuration"
                        })
                        continue

                    matieres_dispensees = {
                        m["matiere"]: niveau_mapping.get(m["niveau"], "Niveau inconnu")
                        for m in tuteur["matieres_dispensees"]
                    }

                    for eleve in tuteur["eleves"]:
                        eleve_prenom = eleve["prenom"]
                        eleve_nom = eleve["nom"]
                        eleve_niveau = niveau_mapping.get(eleve["niveauEleve"], "Niveau inconnu")
                        eleve_niveau_classe = inverse_niveau_mapping.get(eleve_niveau, "Classe inconnue")
                        eleve_matieres = eleve.get("matieres", [])

                        # 🛑 Si l'élève n'a pas de matière, générer une erreur de Configuration
                        if not eleve_matieres or all(matiere.strip() == "" for matiere in eleve_matieres):
                            errors.append({
                                "centre": centre_nom,
                                "date": date,
                                "heure": heure,
                                "eleve": f"{eleve_prenom} {eleve_nom}",
                                "erreur": f"Il manque les matières pour {eleve_prenom} {eleve_nom}.",
                                "type_erreur": "Configuration"
                            })
                            continue  # Ne pas vérifier les matières du tuteur pour cet élève

                        # Si l'élève a "Aide aux devoirs", vérifier seulement Maths et Français
                        matieres_a_verifier = []
                        for matiere in eleve_matieres:
                            if matiere == "Aide aux devoirs":
                                matieres_a_verifier.extend(["Maths", "Français"])
                            else:
                                matieres_a_verifier.append(matiere)

                        for matiere in matieres_a_verifier:
                            if matiere not in matieres_dispensees:
                                errors.append({
                                    "centre": centre_nom,
                                    "date": date,
                                    "heure": heure,
                                    "eleve": f"{eleve_prenom} {eleve_nom}",
                                    "tuteur": f"{tuteur_prenom} {tuteur_nom}",
                                    "erreur": f"{eleve_prenom} {eleve_nom} suit {matiere} avec {tuteur_prenom}, qui ne l'enseigne pas.",
                                    "type_erreur": "Matieres"
                                })
                            else:
                                tuteur_niveau = matieres_dispensees[matiere]
                                tuteur_niveau_classe = inverse_niveau_mapping.get(tuteur_niveau, "Classe inconnue")

                                if tuteur_niveau < eleve_niveau:
                                    errors.append({
                                        "centre": centre_nom,
                                        "date": date,
                                        "heure": heure,
                                        "eleve": f"{eleve_prenom} {eleve_nom}",
                                        "tuteur": f"{tuteur_prenom} {tuteur_nom}",
                                        "erreur": f"{eleve_prenom} {eleve_nom} est en classe de {eleve_niveau_classe} en {matiere}, mais {tuteur_prenom} n'a qu'un niveau {tuteur_niveau_classe}.",
                                        "type_erreur": "Matieres"
                                    })

    return errors

# Exécution des validations
errors_list = validate_schedule(data)

# Sauvegarde des erreurs dans un fichier JSON
output_dir = "outputs"
os.makedirs(output_dir, exist_ok=True)
errors_file = os.path.join(output_dir, "errors.json")

with open(errors_file, "w", encoding="utf-8") as f:
    json.dump({"errors": errors_list}, f, indent=4)

print("✅ Fichier des erreurs sauvegardé :", errors_file)

# Envoi d'un tableau JSON unique à Zapier
zapier_webhook_url = "https://hooks.zapier.com/hooks/catch/12895606/2fwmqm2/"

if errors_list:
    payload = {"errors": errors_list}  # Envoi sous forme d'un seul objet JSON
    try:
        response = requests.post(zapier_webhook_url, json=payload)
        if response.status_code == 200:
            print(f"✅ Toutes les erreurs envoyées à Zapier avec succès !")
        else:
            print(f"❌ Erreur lors de l'envoi à Zapier: {response.status_code} - {response.text}")
    except Exception as e:
        print(f"🚨 Exception lors de l'envoi à Zapier: {e}")
else:
    print("✅ Aucun problème détecté, rien à envoyer à Zapier.")

print("✅ Envoi terminé.")


🔄 Extraction du planning en cours...
🔽 Téléchargement du fichier JSON...
✅ Fichier JSON téléchargé avec succès.
✅ Fichier des erreurs sauvegardé : outputs/errors.json
✅ Toutes les erreurs envoyées à Zapier avec succès !
✅ Envoi terminé.
